In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.utils import shuffle

In [41]:
iris = datasets.load_iris()


X = iris.data[:100, :]
Y = iris.target[:100]
X_test = iris.data[100:, :]
logreg = linear_model.LogisticRegression()

logreg.fit(X, Y)
print(X_test)

[[ 6.3  3.3  6.   2.5]
 [ 5.8  2.7  5.1  1.9]
 [ 7.1  3.   5.9  2.1]
 [ 6.3  2.9  5.6  1.8]
 [ 6.5  3.   5.8  2.2]
 [ 7.6  3.   6.6  2.1]
 [ 4.9  2.5  4.5  1.7]
 [ 7.3  2.9  6.3  1.8]
 [ 6.7  2.5  5.8  1.8]
 [ 7.2  3.6  6.1  2.5]
 [ 6.5  3.2  5.1  2. ]
 [ 6.4  2.7  5.3  1.9]
 [ 6.8  3.   5.5  2.1]
 [ 5.7  2.5  5.   2. ]
 [ 5.8  2.8  5.1  2.4]
 [ 6.4  3.2  5.3  2.3]
 [ 6.5  3.   5.5  1.8]
 [ 7.7  3.8  6.7  2.2]
 [ 7.7  2.6  6.9  2.3]
 [ 6.   2.2  5.   1.5]
 [ 6.9  3.2  5.7  2.3]
 [ 5.6  2.8  4.9  2. ]
 [ 7.7  2.8  6.7  2. ]
 [ 6.3  2.7  4.9  1.8]
 [ 6.7  3.3  5.7  2.1]
 [ 7.2  3.2  6.   1.8]
 [ 6.2  2.8  4.8  1.8]
 [ 6.1  3.   4.9  1.8]
 [ 6.4  2.8  5.6  2.1]
 [ 7.2  3.   5.8  1.6]
 [ 7.4  2.8  6.1  1.9]
 [ 7.9  3.8  6.4  2. ]
 [ 6.4  2.8  5.6  2.2]
 [ 6.3  2.8  5.1  1.5]
 [ 6.1  2.6  5.6  1.4]
 [ 7.7  3.   6.1  2.3]
 [ 6.3  3.4  5.6  2.4]
 [ 6.4  3.1  5.5  1.8]
 [ 6.   3.   4.8  1.8]
 [ 6.9  3.1  5.4  2.1]
 [ 6.7  3.1  5.6  2.4]
 [ 6.9  3.1  5.1  2.3]
 [ 5.8  2.7  5.1  1.9]
 [ 6.8  3.2

In [42]:
logreg.predict([[2.1,1,0.5,10]])

array([1])

In [43]:
import tensorflow as tf
from sklearn import datasets, utils
from sklearn.preprocessing import label_binarize
raw_data_set = datasets.load_iris() 
iris = np.column_stack([raw_data_set['data'], raw_data_set['target']]) 
iris = utils.shuffle(iris) 
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

In [48]:
# Parameters
learning_rate = 0.0011
training_epochs = 30
batch_size = 10
display_step = 1

n_hidden_1 = 8 # 1st layer number of neurons
n_hidden_2 = 8 # 2nd layer number of neurons
n_input = 4 # MNIST data input (img shape: 28*28)
n_classes = 3 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = 150 
        train_x = iris[:100, :4] 
        train_y = target_multiclass[:100]
        total_batch = int(100/batch_size)
        # Loop over all batches
        for i in range(0, iris.data.shape[0],batch_size):
            batch_x,batch_y=train_x[i:i+batch_size],train_y[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[100:150, :4], Y: target_multiclass[100:150]}))

Epoch: 0001 cost=nan
Epoch: 0002 cost=nan
Epoch: 0003 cost=nan
Epoch: 0004 cost=nan
Epoch: 0005 cost=nan
Epoch: 0006 cost=nan
Epoch: 0007 cost=nan
Epoch: 0008 cost=nan
Epoch: 0009 cost=nan
Epoch: 0010 cost=nan
Epoch: 0011 cost=nan
Epoch: 0012 cost=nan
Epoch: 0013 cost=nan
Epoch: 0014 cost=nan
Epoch: 0015 cost=nan
Epoch: 0016 cost=nan
Epoch: 0017 cost=nan
Epoch: 0018 cost=nan
Epoch: 0019 cost=nan
Epoch: 0020 cost=nan
Epoch: 0021 cost=nan
Epoch: 0022 cost=nan
Epoch: 0023 cost=nan
Epoch: 0024 cost=nan
Epoch: 0025 cost=nan
Epoch: 0026 cost=nan
Epoch: 0027 cost=nan
Epoch: 0028 cost=nan
Epoch: 0029 cost=nan
Epoch: 0030 cost=nan
Optimization Finished!
Accuracy: 0.92
